## Install beautifulsoup4 to scrap webpage

In [ ]:
pip install beautifulsoup4

## Import requests package to get the html content of a webpage 
## Then get the html content of the canada postal codes webpage into a variable

In [1]:
import requests
html_content = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

### Parse the content using beautiful soup html parser and print it

In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')
#print(soup.prettify()) # print the parsed data of html

## Print the title of the webpage

In [3]:
print(soup.title)

<title>List of postal codes of Canada: M - Wikipedia</title>


### By doing webpage inspect get the class name of the table and assign that table to a variable
### Get the header of the table using tag "th" and print the headers after removing spaces and new line character


In [30]:
canada_table = soup.find("table", attrs={"class": "wikitable sortable"})

t_headers = []
for th in canada_table.find_all("th"):
 # remove any newlines and extra spaces from left and right
 t_headers.append(th.text.replace('\n', ' ').strip())        
   
    
print(t_headers)


['Postal Code', 'Borough', 'Neighborhood']


## Get all rows from the table's body and assign that to an array

In [5]:
    table_data = []
    for tr in canada_table.tbody.find_all("tr"): # find all tr's from table's tbody
        t_row = {}
        # Each table row is stored in the form of
        # t_row = {'Postal Code': '', 'Borough': '', 'Neighborhood': ''}

        # find all td's(3) in tr and zip it with t_header
        for td, th in zip(tr.find_all("td"), t_headers): 
            t_row[th] = td.text.replace('\n', '').strip()
        table_data.append(t_row)

#Remove first record
table_data.pop(0) 
print(table_data[0])

{'Postal Code': 'M1A', 'Borough': 'Not assigned', 'Neighborhood': 'Not assigned'}


## Import pandas to create data frame

In [6]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


## Define the data frame columns and instantiate the data frame

In [34]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,PostalCode,Borough,Neighborhood


## Get the values form array and assign it to the corresponding columns in the data frame

In [39]:
for data in table_data:
    Borough = data['Borough']
    PostalCode = data['Postal Code']
    Neighborhood = data['Neighborhood']
    
    
    neighborhoods = neighborhoods.append({'Borough': Borough,
                                          'PostalCode': PostalCode,
                                          'Neighborhood': Neighborhood}, ignore_index=True)
neighborhoods.shape

(180, 3)

## Combine rows into one row with the neighborhoods separated with a comma if there are more than one neighborhood for a given Postalcode and Borough

In [ ]:
neighborhoods_grouped = neighborhoods.groupby(['PostalCode', 'Borough']).agg(','.join)
neighborhoods_grouped = neighborhoods_grouped.reset_index()

In [21]:
neighborhoods_grouped.head()


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn


## If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [23]:
neighborhoods_grouped.loc[neighborhoods_grouped['Neighborhood'] == 'Not assigned' , 'Neighborhood'] = neighborhoods_grouped['Borough']
neighborhoods_grouped.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae
6,M1J,Scarborough,Scarborough Village
7,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
8,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
9,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"


## To print the number of rows of dataframe

In [35]:
neighborhoods_grouped.shape

(180, 3)

In [41]:
pip install geocoder

     |████████████████████████████████| 102kB 6.0MB/s ta 0:00:011
  Using cached https://files.pythonhosted.org/packages/d2/3d/fa76db83bf75c4f8d338c2fd15c8d33fdd7ad23a9b5e57eb6c5de26b430e/click-7.1.2-py2.py3-none-any.whl
     |████████████████████████████████| 829kB 6.9MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [68]:
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [70]:
Geospatial_Coordinates = pd.read_csv('Geospatial_Coordinates.csv', skiprows = 1)
Geospatial_Coordinates.columns=["PostalCode","Latitude","Longitude"]
Geospatial_Coordinates

,PostalCode,Latitude,Longitude
0,M1C,43.784535,-79.160497
1,M1E,43.763573,-79.188711
2,M1G,43.770992,-79.216917
3,M1H,43.773136,-79.239476
4,M1J,43.744734,-79.239476
5,M1K,43.727929,-79.262029
6,M1L,43.711112,-79.284577
7,M1M,43.716316,-79.239476
8,M1N,43.692657,-79.264848
9,M1P,43.757410,-79.273304


In [71]:
merged_inner = pd.merge(left=neighborhoods_grouped, right=Geospatial_Coordinates, left_on='PostalCode', right_on='PostalCode')
merged_inner.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
1,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
2,M1G,Scarborough,Woburn,43.770992,-79.216917
3,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
4,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
5,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
6,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
7,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
8,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
9,M1P,Scarborough,"Dorset Park, Wexford Heights, Scarborough Town...",43.757410,-79.273304
